# 7. 가설과 추론

## 8.26(토) 신성진

Goal: 지금까지 배웠던 통계와 확률 이론으로 가설을 검정하고 통계적으로 추론을 해 보자

7.1 통계적 가설검정

가설(hypothesis)의 예: 동전은 앞뒤가 나올 확률이 공평하다 / 데이터과학자는 알보다 파이썬을 선호한다 등의 주장

H0 vs. H1
기본적인 가설을 의미하는 귀무가설 (H0, Null Hypothesis)와 비교하고 싶은 대립가설 (H1, alternative hypothesis)
통계를 사용해서 H0을 기각할지 말지를 결정

In [11]:
from probability import normal_cdf, inverse_normal_cdf # 이전장에서 작업한 확률 코드를 임포트 해야함
import math, random
from __future__ import division

7.2 동전 던지기 예시

동전이 하나 있는데, 공평한 동전인지 아닌지 검정하고 싶음.

H0: p = 0.5 vs H1: p != 0.5

가정: 확률변수가 정규분포를 따른다는 가정

In [12]:
def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [13]:
# 누적분포함수는 확률변수가 특정 값보다 작은 확률 나타냄
normal_probability_below = normal_cdf

# it's above the threshold if it's not below the threshold
# 만약 확률변수가 특정 값보다 작지 않다면, 특정 값보다 크다는 것을 의미
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)
    
# it's between if it's less than hi, but not less than lo
# 만약 확률변수가 hi보다 작고 lo보다 작지 않다면, 확률변수는 hi와 low사이에 존재
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# it's outside if it's not between
# 만약 확률 변수가 범위 밖에 존재한다면, 범위 안에 존재하지 않는것을 의미
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

"""반대로 확률이 주어졌을 때, 평균을 중심으로 하는 (대칭) 구간을 구할 수 있음.
예를 들어, 분포의 60%를 차지하는 평균 중심의 구간을 구하고 싶으면, 양쪽 꼬리 부분이 각각 분포의 20%차지하는 부분을 구하면 됨"""

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)
    
def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds 
    that contain the specified probability"""
    tail_probability = (1 - probability) / 2

    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

In [14]:
#실제로 동전을 1000번 던져보기. 동전이 공평하면 X = 500 표준편차는 15.8

mu_0, sigma_0 = normal_approximation_to_binomial(1000,0.5)

mu_0, sigma_0

(500.0, 15.811388300841896)

1종 오류를 얼마나 허용해 줄 것인지를 의미하는 유의수준(sig) 결정

![Image of Errors](http://data.newdaily.co.kr/data/photos/20150939/shp_1443320553.jpg)

In [15]:
#Significant를 0.05로 설정, p가 참이면 X가 주어진 범위에서 벗어날 확률은 5%이하 = 20번중에 19은 맞는 결과

normal_two_sided_bounds(0.95, mu_0, sigma_0) 

(469.01026640487555, 530.9897335951244)

In [16]:
#2종오류를 범하지 않을 확률을 구하려면 검정력 (power)를 알 수 있음
#2종오류는 H0이 거짓이지만 기각하지 않을때

#p가 0.5라고 가정 할 때, 유의수준이 5%인 구간
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

#p가 0.55인 경우의 실제 평균과 표준편차
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

#제2종 오류는, 귀무가설 (h0)를 기각하지 못한다는 의미, 즉 X가 구간 안에 존재
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)

power = 1 - type_2_probability

print(power)

0.886548001295


한편, p <= 0.5, 즉 동전이 앞면에 편향되지 않는 경우를 귀무가설로 정하면 X가 50보다 크면 귀무가설을 기각하고 50보다 작으면 기각하지 않는 단측 검증 필요

유의수준이 5%인 가설검정을 위해서는 normal_probability_below를 사용하여 분포의 95%가 해당값 이하는 경계 값을 찾을 수 있다.

In [17]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
print(hi) #( < 531, 분포 상위 부분에 더 높은 확률을 주기 위해)

526.007358524


In [18]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

0.936379480331


이 가설검정은 X가 469보다 작을 때 H0를 기각하는게 아니라(H1이 참이면
이는 거의 발생 안함) X가 526~531 사이 일 때,
H0를 기각하기 때문에 (H1이 참이라면 이는 가끔 발생)
전보다 검정력이 더 좋아 졌다고 볼 수 있다.

## 7.3 P-value
어떤 확률 값을 기준으로 구간을 선택하는 대신, H0가 참이라고 가정하고 실제로 관측된 값보다 더 극단적인 값이 나올 확률을 구하는 것임.

동전이 공평한지 판단하기 위해 양측 검증을 하자

In [19]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is above x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is below x
        return 2 * normal_probability_below(x, mu, sigma)   

In [20]:
#만약 동전의 앞면의 530번 관측되면, p-value는 아래와 같음
# 530보다 529.5를 사용한 것은 연속수정 (continuity correction* ?)일 때임


two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598857

In [21]:
def count_extreme_values():
    extreme_value_count = 0
    for _ in range(100000):
        num_heads = sum(1 if random.random() < 0.5 else 0    # count # of heads
                        for _ in range(1000))                # in 1000 flips
        if num_heads >= 530 or num_heads <= 470:             # and count how often
            extreme_value_count += 1                         # the # is 'extreme'

    return extreme_value_count / 100000

In [22]:
count_extreme_values() #0.062
# p값이 0.05보다 크게 나왔기 때문에 H0를 기각하지 않는다. 만약에 동전의 앞면이 532번
# 나왔으면 0.05보다 작고, 이 경우는 귀무 가설을 기각

0.06169

7.4 신뢰구간

분포를 모른다면, Confidence Interval을 사용하여 가설 검증 가능.


In [23]:
p = 0.525

math.sqrt(p * (1-p) / 1000)

0.015791611697353755

In [24]:
#p의 정확한 값을 모르므로 추정치를 사용 가능

p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

lower, upper = normal_two_sided_bounds(0.95, mu, sigma)

lower < 0.5  < upper

True

In [25]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

lower, upper = normal_two_sided_bounds(0.95, mu, sigma)

lower < 0.5  < upper

False

In [26]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [27]:
def run_experiment():
    """flip a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    """using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

In [28]:
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]

[sum(experiment) for experiment in experiments][:10]

[508, 491, 484, 483, 502, 499, 483, 511, 481, 538]

In [29]:
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])
num_rejections

46

# 7.6 A,B Testing

## A/B Testing은 전체 디자인에서 한가지 요소에 대한 두 가지 이상의 버전을 시험하여 더 나은 것을 판별하는 기법입니다. 보통은 기존의 버전(A)과 새로운 버전(B)를 가지고 랜덤하게 방문하는 사용자 별로 다른 버전을 보여준 후, 의도하는 결과가 높게 나오는 쪽이 어느 쪽인지를 검증해나갑니다.



# 왜 하나

A/B 테스팅을 하는 이유는 상관관계로부터 인과관계 - 정확히 말하면 인과관계일 가능성이 높은 것 - 를 찾아내기 위함이다. 그래야만 우리가 “원인”에 해당하는 요소에 개입을 하여 “결과”에 해당하는 요소가 우리가 원하는 방향으로 변화되도록 할 수 있다. 혹은 이미 “결과”에 변화가 생겼을 때 이 변화의 “원인”이 우리가 했던 그 개입 때문이 맞는지 아닌지 판단할 수 있다.

## 잘못 된 점

물놀이 사고를 줄이는 것이 우리의 목표라고 가정하자. 이런저런 조사를 해보니 아이스크림 판매량과 물놀이 사고의 빈도 사이에 높은 양의 상관이 있다는 사실을 찾아냈다. 아이스크림 판매량이 높아지면 물놀이 사고 빈도도 높아지고, 아이스크림 판매량이 낮아지면 물놀이 사고 빈도도 낮아지는 식이다.

이 상관관계를 아래와 같은 인과관계로 해석하면 물놀이 사고를 줄이기 위해 우리가 해야할 일은 명확하다. 아이스크림 가격을 올리는 등 다양한 방법을 써서 아이스크림의 판매량을 줄이는 것이다:

"아이스크림 판매량" -> "물놀이 사고 빈도"
하지만 모든 상관관계가 인과관계인 것은 아니다. “Correlation does not imply causation”이라는 유명한 문구가 바로 위와 같은 상황을 이르는 말이다.

![Correlation](http://imgs.xkcd.com/comics/correlation.png)

"날씨" -> {"아이스크림 판매량" "물놀이 사고 빈도"}


## 예시

좀 더 현실적인 예를 떠올려보자. 어떤 쇼핑몰 웹 사이트가 있다. 3개월에 걸쳐 디자인 개편 프로젝트를 진행하였고 지난주에 성공적으로 새 디자인을 적용하였다. 그랬더니 갑자기 그 전에 비해 일 매출이 10% 증가했다.

매출 증가가 디자인 덕분이라고 할 수 있을까?

"디자인 개편" -> "매출 증가"

직전의 아이스크림 예시에서 살펴본 바와 같이 두 사건이 함께 일어난다고 해서 하나의 사건이 다른 사건의 원인이라고 가정하는 것은 위험하다. 예를 들면 새 디자인이 적용된 바로 그 날, 하필이면 경쟁 쇼핑몰이 문을 닫았을 수도 있다. 이 경우 매출 증가의 원인은 외부 요인에 있다. 또는 새 디자인이 적용된 바로 그 날, 경쟁력 있는 상품이 입고되었을 수도 있다. 이 경우 매출 증가의 원인은 내부에 있으나 그 성과는 디자인팀이 아니라 영업팀 덕으로 돌려야 한다. 심지어는 갑자기 경기가 좋아졌기 때문일 수도 있다. 다른 쇼핑몰들을 일 매출이 15% 씩 향상되었는데 만약 우리 쇼핑몰만 10% 향상된 것이라면 새 디자인이 매출에 악영향을 준 것일지도 모른다.

A/B 테스팅을 하면 방문자를 “임의로” A, B 두 집단으로 나누고 A 집단의 방문자에게는 기존 디자인을 보여주고, B 집단의 방문자에게만 새 디자인을 보여준다.

새 디자인을 적용하기 전의 매출과 새 디자인을 적용한 후의 매출을 비교하는 대신, A 집단과 B 집단의 매출을 비교하면 시간의 흐름에 따라 발생하는 다른 요인들(경쟁 쇼핑몰 부도, 신상품 입고, 경기 변동 등)을 통제할 수 있기 때문에 순수하게 디자인 변화로 인한 매출 차이만을 가려낼 수 있다. A 집단에 비해 B 집단에서 매출 증가가 있었다면 “디자인 개편”과 “매출 증가” 사이에는 인과관계가 성립할 가능성이 대단히 크다고 말할 수 있다.

# 주의할 점들

A/B 테스팅을 통해 인과관계를 찾아내려면 두 집단을 임의적으로 나누어야 한다. 이를 임의적 할당(random assignment)이라고 한다. 예를 들어

남성은 A 집단, 여성은 B 집단
짝수 시간대 방문자는 A 집단, 홀수 시간대 방문자는 B 집단
첫 일주일 동안 방문한 사용자는 A 집단, 그 다음 일주일 동안 방문한 사용자는 B 집단
등 임의적이지 않은 방식을 사용할 경우 두 집단의 차이가 무엇 때문에 발생하는지 가려낼 수 없게 된다.

A/B 테스팅을 통해 찾아낸 결과가 범용성을 지니려면 애초에 실험에 참가한 집단이 모집단을 대표할 수 있어야한다. 이를 임의적 추출(random sampling)이라고 한다. 예를 들어 초등학교 학생들을 대상으로 한 실험의 결과를 초중고등학교 학생 모두에게 적용하거나, 페이스북 사용자를 대상으로 한 실험의 결과를 트위터에 적용하거나 하면 추출된 집단의 성격과 모집단의 성격에 차이가 있기 때문에 기대와 다른 결과가 나올 수 있다.



# A/B 테스팅의 단점

## 첫째, 테스트를 많이/자주하면 단기적으로 손해가 발생할 수 있다.

예를 들어 쇼핑몰에서 구매전환율이 높은 상품 이미지가 무엇인지 알아보기 위한 테스팅을 진행한다고 치자. 2주 동안 전체 방문자를 50:50으로 나눠 기존 상품 이미지와 새로운 상품 이미지를 보여주고자 한다. 테스트를 3일 쯤 진행했더니 새로운 이미지를 본 집단에서의 매출이 기존에 비해 절반 밖에 나오지 않는다면 어떻게 해야하나? 애초에 계획했던 2주동안 테스트를 진행하려면 막심한 매출 손해를 감수해야한다.

## 둘째, A/B 테스팅의 결과는 계절 변화나 취향 변화 등 시간의 흐름에 따라 바뀔 수 있다. 

작년 겨울에 A/B 테스팅을 하여 얻은 결론은 언제까지 유효할까? 통제실험은 시공간의 보편성에 대한 가정을 깔고 있다. 이 가정은 물리학이나 화학 수준에서는 대단히 확실히 보장되고, 생물학을 거쳐 사회과학 분야로 가면서 점점 약해지며, 비즈니스 맥락에서는 대단히 약해진다. 어제의 세상과 오늘의 세상이 다르고, 미국과 한국이 다르다. 결국 확실성을 유지하기 위해서는 실험을 지속적으로 반복해서 해야하는데 첫번째 단점(비용 문제)과 엮어서 생각해본다면 곤란한 얘기가 된다.

## 셋째, A/B 테스팅만 해서는 지역최적점에 머물게 될 위험이 있다. 

A/B 테스팅이라는 것은 기존 상태에서 작은 변화(되도록 하나의 변수만 살짝 바꾸기)를 가하며 점진적으로 더 나은 상태를 찾아가는 방식으로 진행된다. 하지만 이 방식으로는 지역최적점에 수렴할 수 있을 뿐 전역적인 최적점을 찾을 수 없다.

이 중 첫번째와 두번째 문제에 대해서는 Multi-armed Bandit 알고리즘이라고 불리는 효과적인 해결책이 있다.
관련 설명: http://www.ecogwiki.com/A-B_Testing%EC%9D%98_%EB%8B%A8%EC%A0%90%EA%B3%BC_%EA%B0%9C%EC%84%A0%EC%95%88

# 코드 예제

## 광고 예제 - 음료광고

A: 맛이 끝내줘요!

B: 편견이 없어요!

In [31]:
#사실상 표준편차를 모르기 때문에 T 분포를 사용해야합니다.
#주어진 데이터가 많으면 차이가 크지 않기 때문에 정규분포를 사용해도 상관없다고 함.

def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [32]:
def B(alpha, beta):
    """a normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:          # no weight outside of [0, 1]    
        return 0        
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

In [33]:
# 1000명중 200명은 맛이 끝내줘요 / 1000명중 180명은 편견이 없어요

z = a_b_test_statistic(1000, 200, 1000, 180)
z

-1.1403464899034472

In [34]:
two_sided_p_value(z) # 두 광고 효과의 평균이 같을 확률이 높음 (p > 0.05) = 광고효과가 같음

0.254141976542236

In [35]:
z = a_b_test_statistic(1000, 200, 1000, 150)
z

-2.948839123097944

In [36]:
two_sided_p_value(z) # 두 광고 효과의 평균이 같을 확률이 매우 낮음 (p < 0.05) = 광고효과가 다름

0.003189699706216853

## 7.7 베이지안 추론

In [37]:
##
#
# Bayesian Inference
#
##

def B(alpha, beta):
    """a normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:          # no weight outside of [0, 1]    
        return 0        
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

In [38]:
beta_pdf(500, 0.1, 0.2)

0

In [9]:
# http://ddiri01.tistory.com/234
# https://github.com/zedoul/Bayesian-for-Hackers-Korean


# 베이지안 통계학의 목표

주어진 데이터(주관적인 견해-prior probability와 이후 시행된 확률실험으로부터 얻어진 결과)를 종합하여 (관심의 대상이 되는) 파라미터의 불확실성을 확률-posterior probability로 나타내는 것

# 베이지안 통계학의 기본 구조

파라미터의 prior distribution을 결정

데이터와 prior distribution을 이용한 posterior distribution 계산

Posterior distribution을 이용한 파라미터 추론

베이지안 통계학에서 파라미터의 prior distribution을 구하는 방법은 여러 가지 문제점이 있는데, 그 중 하나는 분석자의 특성에 따라 prior distribution이 달라서 상당히 주관적이라는 것이다. 그래서 객관적인 prior distribution이 제안되었는데 한 예가 noninformative prior distribution으로 Laplace에 의해 제안된 uniform distribution을 이용하는 것이다. 아무런 정보가 없는 상태에서는 모든 parameter들의 값은 동일하게 취급되어야 한다는 것인데 이론적으로 문제(lack of invariance for transformation)가 있음이 밝혀졌고 그 대안 중의 하나가 Jeffrey의 prior distribution이다.

## 베이지안 통계학의 장점

결과에 대한 해석이 전통적인 통계학보다 훨씬 쉽고 우리의 직관과 잘 부합된다.

통계학의 목표는 미지의 parameter에 대해 추론하고 그 추론의 불확실성을 계량화하는 것이다. 그런데 불확실성을 나타내는 가장 좋은 방법이 probability이며, 베이지안 통계학은 probability를 직접 이용하는 통계적 기법이다.

## 실제로 많은 문제들에서 parameter의 prior information을 쉽게 구할 수 있다.

전통적인 통계학에서는 공통분산을 사용하는데 이는 prior information을 굉장히 단순하게 사용한 것이다. 베이지안 통계학은 더욱 효과적으로 prior 
information을 이용한 추론 결과를 제시할 수 있다.

## 베이지안 기법이 전통적인 통계학의 관점에서도 좋은 추론 방법을 제공한다.

전통적인 통계학에서 사용되는 많은 통계량들은 베이지안 추정량 또는 베이지안 추정량의 극한으로 구할 수 있다.

## 복잡한 문제에서 parameter의 추론이 전통적인 통계학보다 쉽다.

전통적인 통계학에서 많이 쓰이는 MLE(Maximum Likelihood Estimator)는 likelihood function이 복잡하거나 많은 local optima를 가지는 경우가 많아 estimation이 어려울 뿐 아니라 일반적으로 estimation 결과가 나쁠 때가 많다. 반면, 베이지안 통계학에서 개발된 MCMC기법을 사용하면 parameter의 prior distribution을 쉽게 구할 수 있을 뿐 아니라, posterior distribution을 통한 parameter 추론을 더욱 정확하게 할 수 있다.
